<a href="https://colab.research.google.com/github/RomanEngeler1805/cohere-hackathon-Sep22/blob/main/CohereHack_Embed_viewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install cohere umap-learn altair annoy datasets tqdm bertopic transformers datasets streamlit pyngrok -q 

In [14]:
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
from sklearn.cluster import KMeans
from bertopic._ctfidf import ClassTFIDF
from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [142]:
!pip install svglib -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 913 kB 4.4 MB/s 
     |████████████████████████████████| 2.8 MB 39.9 MB/s 
     |████████████████████████████████| 3.1 MB 44.8 MB/s 
  Created wheel for svglib: filename=svglib-1.4.1-py3-none-any.whl size=30771 sha256=14e55acc9edcccf1fef16f9cb05e2203908592e01e48f901c6f3710b24dee4e1
  Stored in directory: /root/.cache/pip/wheels/97/77/25/3980452b372d8c3b8115fedfb0caa1e3d7efaf251a11a9f45b
Successfully built svglib
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [3]:
s = load_dataset('snli')
s['train'][15:20]

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'premise': ['A few people in a restaurant setting, one of them is drinking orange juice.',
  'A few people in a restaurant setting, one of them is drinking orange juice.',
  'A few people in a restaurant setting, one of them is drinking orange juice.',
  'An older man is drinking orange juice at a restaurant.',
  'An older man is drinking orange juice at a restaurant.'],
 'hypothesis': ['The people are eating omelettes.',
  'The people are sitting at desks in school.',
  'The diners are at a restaurant.',
  'A man is drinking juice.',
  'Two women are at a restaurant drinking wine.'],
 'label': [1, 2, 0, 0, 2]}

In [4]:
from datasets import load_dataset

def get_dataset(df, text, title):
  df.rename(columns={text: 'text', title: 'title'}, inplace=True)
  df = df[['title', 'text']]
  max_length = max(1000, df.shape[0])
  df = df.head(max_length)
  return df

s = load_dataset('snli')
s.set_format('pandas')
df = s['train'][:]

df = get_dataset(df, text='premise', title='hypothesis')
df.head(3)

  0%|          | 0/3 [00:00<?, ?it/s]

,title,text
0,A person is training his horse for a competition.,A person on a horse jumps over a broken down airplane.
1,"A person is at a diner, ordering an omelette.",A person on a horse jumps over a broken down airplane.
2,"A person is outdoors, on a horse.",A person on a horse jumps over a broken down airplane.


## Basic plot

In [5]:
api_key = 'dwhPny8kTpkhDNpu05484MtqjFU2QKXeYx9kH6DA'
co = cohere.Client(api_key)
title = 'Some title needs to be inputted here'

def get_dataset(df, text, title):
  max_length = 1000
  df.rename(columns={text: 'text', title: 'title'}, inplace=True)
  df = df[['title', 'text']]
  max_length = min(max_length, df.shape[0])
  df = df.head(max_length)
  return df

s = load_dataset('snli')
s.set_format('pandas')
df = s['train'][:]


def get_embeddings(df):
  embeds = co.embed(texts=list(df['text']),
                  model='large',
                  truncate='LEFT').embeddings
  reducer = umap.UMAP(n_neighbors=100) 
  umap_embeds = reducer.fit_transform(embeds)
  return (embeds, umap_embeds)

#Function calls
df = get_dataset(df, text='premise', title='hypothesis')
embeds, umap_embeds = get_embeddings(df)


df['x'] = umap_embeds[:,0]
df['y'] = umap_embeds[:,1]

# Plot
chart = alt.Chart(df).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    tooltip=['title']
).configure(background="#FDF7F0"
).properties(
    width=700,
    height=400,
    title=title
)

chart.interactive()


  0%|          | 0/3 [00:00<?, ?it/s]

alt.Chart(...)

## Cluster plot

In [76]:
api_key = 'WKsYz7mFBHEDsI7xsZ9KJP3WPMm5txIjKmU0eBTK'
co = cohere.Client(api_key)
title = 'Some title needs to be inputted here'

def get_dataset(df, text, title):
  max_length = 1000
  df.rename(columns={text: 'text', title: 'title'}, inplace=True)
  df = df[['title', 'text']]
  max_length = min(max_length, df.shape[0])
  df = df.head(max_length)
  return df

s = load_dataset('snli')
s.set_format('pandas')
df = s['train'][:]

def get_keywords(df, n_clusters=8, chart_title='This is the title'):
  def get_embeddings(df):
    embeds = co.embed(texts=list(df['text']),
                    model='large',
                    truncate='LEFT').embeddings
    reducer = umap.UMAP(n_neighbors=100) 
    umap_embeds = reducer.fit_transform(embeds)
    return (embeds, umap_embeds)

  embeds, umap_embeds = get_embeddings(df)
  df['x'] = umap_embeds[:,0]
  df['y'] = umap_embeds[:,1]

  kmeans_model = KMeans(n_clusters=n_clusters, random_state=0)
  classes = kmeans_model.fit_predict(embeds)
  documents =  df['title']
  documents = pd.DataFrame({"Document": documents,
                            "ID": range(len(documents)),
                            "Topic": None})
  documents['Topic'] = classes
  documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
  count_vectorizer = CountVectorizer(stop_words="english").fit(documents_per_topic.Document)
  count = count_vectorizer.transform(documents_per_topic.Document)
  words = count_vectorizer.get_feature_names()
  ctfidf = ClassTFIDF().fit_transform(count).toarray()
  words_per_class = {label: [words[index] for index in ctfidf[label].argsort()[-10:]] for label in documents_per_topic.Topic}
  df['cluster'] = classes
  df['keywords'] = df['cluster'].map(lambda topic_num: ", ".join(np.array(words_per_class[topic_num])[:]))
  return df, chart_title

#Function calls
df = get_dataset(df, text='premise', title='hypothesis')
df, chart_title = get_keywords(df, n_clusters=8, chart_title='This is the title')


# Plot
selection = alt.selection_multi(fields=['keywords'], bind='legend')

chart = alt.Chart(df).transform_calculate(
    url='https://news.ycombinator.com/item?id=' + alt.datum.id
).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    href='url:N',
    color=alt.Color('keywords:N', 
                    legend=alt.Legend(columns=1, symbolLimit=0, labelFontSize=14)
                   ),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['title', 'keywords', 'cluster']
).properties(
    width=800,
    height=500
).add_selection(
    selection
).configure_legend(labelLimit= 0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0").properties(
    title=chart_title
)
chart.interactive()

  0%|          | 0/3 [00:00<?, ?it/s]

alt.Chart(...)

In [ ]:
## SNLI dataset
s = load_dataset('snli')
s.set_format('pandas')
df = s['train'][:]

#Function calls
df = get_dataset(df, text='premise', title='hypothesis')
df, chart_title = get_keywords(df, n_clusters=8, chart_title='This is the title')

# Plot
selection = alt.selection_multi(fields=['keywords'], bind='legend')

chart = alt.Chart(df).transform_calculate(
    url='https://news.ycombinator.com/item?id=' + alt.datum.id
).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    href='url:N',
    color=alt.Color('keywords:N', 
                    legend=alt.Legend(columns=1, symbolLimit=0, labelFontSize=14)
                   ),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['title', 'keywords', 'cluster']
).properties(
    width=800,
    height=500
).add_selection(
    selection
).configure_legend(labelLimit= 0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0").properties(
    title=chart_title
)
chart.interactive()

  0%|          | 0/3 [00:00<?, ?it/s]

alt.Chart(...)

In [77]:
df.head(5)

,title,text,x,y,cluster,keywords
0,A person is training his horse for a competition.,A person on a horse jumps over a broken down airplane.,14.873435,2.603458,0,"juice, chess, motorcycle, guitar, computer, looking, sits, playing, sitting, man"
1,"A person is at a diner, ordering an omelette.",A person on a horse jumps over a broken down airplane.,14.890037,2.567093,0,"juice, chess, motorcycle, guitar, computer, looking, sits, playing, sitting, man"
2,"A person is outdoors, on a horse.",A person on a horse jumps over a broken down airplane.,14.923051,2.593711,0,"juice, chess, motorcycle, guitar, computer, looking, sits, playing, sitting, man"
3,They are smiling at their parents,Children smiling and waving at camera,2.638442,5.184999,2,"girls, outside, group, children, field, cheerleaders, soccer, family, playing, people"
4,There are children present,Children smiling and waving at camera,2.628999,5.193110,2,"girls, outside, group, children, field, cheerleaders, soccer, family, playing, people"


In [74]:
# Amazon review dataset
def filter_dataset(row):
  return row['language'] == 'en'

a = load_dataset('amazon_reviews_multi')
a = a.filter(filter_dataset)
a.set_format('pandas')
df = a['train'][:]

#Function calls
df = get_dataset(df, text='review_body', title='review_title')
df, chart_title = get_keywords(df, n_clusters=8, chart_title='This is the title')

# Plot
selection = alt.selection_multi(fields=['keywords'], bind='legend')

chart = alt.Chart(df).transform_calculate(
    url='https://news.ycombinator.com/item?id=' + alt.datum.id
).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    href='url:N',
    color=alt.Color('keywords:N', 
                    legend=alt.Legend(columns=1, symbolLimit=0, labelFontSize=14)
                   ),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['title', 'keywords', 'cluster']
).properties(
    width=800,
    height=500
).add_selection(
    selection
).configure_legend(labelLimit= 0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0").properties(
    title=chart_title
)
chart.interactive()

  0%|          | 0/3 [00:00<?, ?it/s]

NameError: ignored

## Semantic Search & Clustering


In [ ]:
# BASICS
# - load / get dataset
# - get cohere embeddings
# - dim reduction embeddings (umap, pca, tsne)

# SEMANTIC SEARCH
# - embed search query
# - similarity measure between query embedding and archive
# - nearest neighbor search
# - plotting

# CONSIDERATIONS
# - handle too long text -> split up and index smart

In [90]:
# Settings
api_key = 'dwhPny8kTpkhDNpu05484MtqjFU2QKXeYx9kH6DA'
title = 'Semantic Search'

co = cohere.Client(api_key)

### Dataset

In [5]:
# Load & Prepare dataset
def get_dataset(df, text, title):
  max_length = 1000
  df.rename(columns={text: 'text', title: 'title'}, inplace=True)
  df = df[['title', 'text']]
  max_length = min(max_length, df.shape[0])
  df = df.head(max_length)
  return df

In [6]:
def filter_dataset(row):
  return row['language'] == 'en'

In [93]:
a = load_dataset('amazon_reviews_multi')
a = a.filter(filter_dataset)
a.set_format('pandas')
df = a['train'][:]

#
df = get_dataset(df, text='review_body', title='review_title')

  0%|          | 0/3 [00:00<?, ?it/s]

In [94]:
df.head(5)

,title,text
0,I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review ...,"Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process."
1,Not use able,the cabinet dot were all detached from backing... got me
2,The product is junk.,I received my first order of this product and it was broke so I ordered it again. The second one was broke in more places than the first. I can't blame the shipping process as it's shrink wrapped and boxed.
3,Fucking waste of money,"This product is a piece of shit. Do not buy. Doesn't work, and then I try to call for customer support, it won't take my number. Fucking rip off!"
4,bubble,went through 3 in one day doesn't fit correct and couldn't get bubbles out (better without)


### Embeddings

In [12]:
# Cohere & Dimensionalit reduction Embeddings
# TODO: class such that reducer state can be persistet
def get_embeddings(df):
  embeds = co.embed(texts=list(df['text']),
                  model='large',
                  truncate='LEFT').embeddings
  reducer = umap.UMAP(n_neighbors=100) 
  umap_embeds = reducer.fit_transform(embeds)
  return (embeds, umap_embeds)

In [96]:
# Get embeddings
embeds, umap_embeds = get_embeddings(df)
df['x'] = umap_embeds[:,0]
df['y'] = umap_embeds[:,1]

### Semantic Search

In [ ]:
# resource: https://docs.cohere.ai/semantic-search

In [97]:
# query and embedding
query = 'which products are broken?'

temp_dict = {'text': query}
df_query = pd.DataFrame(temp_dict, index=[0])

# embed query
query_embed, query_umap_embed = get_embeddings(df_query)

In [98]:
# create search index
embeds = np.array(embeds)

search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

In [99]:
similar_item_ids

([2, 820, 175, 662, 861, 365, 979, 959, 316, 396],
 [0.9387723803520203,
  0.9695602059364319,
  0.9843848347663879,
  0.9874251484870911,
  0.992482602596283,
  0.9944567680358887,
  1.0063396692276,
  1.0066001415252686,
  1.009747862815857,
  1.014773964881897])

In [100]:
# Retrieve the nearest neighbors
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                 include_distances=True)
# Format the results
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]})
results

,texts,distance
2,I received my first order of this product and it was broke so I ordered it again. The second one was broke in more places than the first. I can't blame the shipping process as it's shrink wrapped and boxed.,0.938772
820,Received broken and missing parts,0.969560
175,Product damaged (chipped glass) when washing on top dishwasher shelf after only a few weeks. Inferior materials and product. I am very displeased with the quality of this item. This happened to several items in our set so not a “one off” defect.,0.984385
662,Every time I placed my order item were delivered broken,0.987425
861,"After taking quite a while to be delivered, what I received was an obviously used, damaged table with a broken equipment. As you can see from the pictures, the legs are scraped and damaged with 4- 6 inch scrapes on both sides and the mechanical pieces that are supposed to hold the table top on are snapped entirely off. You can see these laying on my counter in the photograph. They are supposed to be attached to the bottom of the table top, which you can see in a separate photo. They sent me broken junk which was likely previously returned which either I must accept, or go through the hassle of sending all the way back to them and cross my fingers that I get another one that’s actually not damaged.",0.992483
365,"Used for a while, broken quality problems",0.994457
979,Item came broken and smashed.,1.006340
959,Cheap. Broke really easy. Half of them were broken when I received them,1.006600
316,"Item received was broken, with product leaking out and all over the jar.",1.009748
396,"Just received these and one is broken. I went to file a return, and they don’t replace it accept returns!!!!! So disappointed!",1.014774


In [101]:
print(results['distance'].values[0])
print(results['distance'].values[-1])

0.9387723803520203
1.014773964881897


### Plot

In [120]:
# dataframe for plotting -> (x, y, neighbour) with neighbour 0 or 1

# TODO: not so elegant
neighbour = []

for k in range(len(df)):
  if k in similar_item_ids[0]:
    neighbour.append(1)
  else:
    neighbour.append(0)


In [121]:
df_neighbour = pd.DataFrame(neighbour, columns=['neighbour'])

In [122]:
df = df.join(df_neighbour);

In [118]:
df.head(5)

,title,text,x,y,neighbour
0,I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review ...,"Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process.",-2.607087,8.266518,0
1,Not use able,the cabinet dot were all detached from backing... got me,-1.630714,8.507143,0
2,The product is junk.,I received my first order of this product and it was broke so I ordered it again. The second one was broke in more places than the first. I can't blame the shipping process as it's shrink wrapped and boxed.,-2.112373,8.170934,2
3,Fucking waste of money,"This product is a piece of shit. Do not buy. Doesn't work, and then I try to call for customer support, it won't take my number. Fucking rip off!",-0.922556,6.293752,0
4,bubble,went through 3 in one day doesn't fit correct and couldn't get bubbles out (better without),-0.863662,8.303803,0


In [125]:
# Plot
chart = alt.Chart(df).transform_calculate(
    url='https://news.ycombinator.com/item?id=' + alt.datum.id
).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    color=alt.Color('neighbour', scale=alt.Scale(domain=[0, 1], range=['blue', 'red'])),
    tooltip=['title']
).properties(
    width=800,
    height=500
).configure_legend(labelLimit= 0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0")

chart.interactive()

alt.Chart(...)

### Combine in one Script

In [16]:
api_key = 'dwhPny8kTpkhDNpu05484MtqjFU2QKXeYx9kH6DA'
co = cohere.Client(api_key)
title = 'Some title needs to be inputted here'

# Get dataset and put into dataframe
s = load_dataset('snli')
s.set_format('pandas')
df = s['train'][:]

df = get_dataset(df, text='premise', title='hypothesis')

# Get embeddings
embeds, umap_embeds = get_embeddings(df)
df['x'] = umap_embeds[:,0]
df['y'] = umap_embeds[:,1]

# query and embedding
query = 'which products are broken?'

temp_dict = {'text': query}
df_query = pd.DataFrame(temp_dict, index=[0])

# embed query
query_embed, query_umap_embed = get_embeddings(df_query)

# create search index
embeds = np.array(embeds)

search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

# Retrieve the nearest neighbors
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                 include_distances=True)
# Format the results
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]})

# find neighbours
neighbour = []

for k in range(len(df)):
  if k in similar_item_ids[0]:
    neighbour.append(1)
  else:
    neighbour.append(0)

df_neighbour = pd.DataFrame(neighbour, columns=['neighbour'])

df = df.join(df_neighbour);

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
# Plot
chart = alt.Chart(df).transform_calculate(
    url='https://news.ycombinator.com/item?id=' + alt.datum.id
).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    color=alt.Color('neighbour', scale=alt.Scale(domain=[0, 1], range=['blue', 'red'])),
    tooltip=['title']
).properties(
    width=800,
    height=500
).configure_legend(labelLimit= 0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0")

chart.interactive()

alt.Chart(...)

# Streamlit App

In [72]:
%%writefile helper.py
import streamlit as st
import pandas as pd
import cohere
import umap
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from sklearn.cluster import KMeans
from bertopic._ctfidf import ClassTFIDF
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

api_key = 'WKsYz7mFBHEDsI7xsZ9KJP3WPMm5txIjKmU0eBTK'
co = cohere.Client(api_key)
title = 'Some title needs to be inputted here'

@st.cache(allow_output_mutation=True)
def get_dataset(df, text, title):
  max_length = 200
  df.rename(columns={text: 'text', title: 'title'}, inplace=True)
  df = df[['title', 'text']]
  max_length = min(max_length, df.shape[0])
  df = df.head(max_length)
  return df

@st.cache(allow_output_mutation=True)
def get_embeddings(df):
    embeds = co.embed(texts=list(df['text']),
                    model='large',
                    truncate='LEFT').embeddings
    reducer = umap.UMAP(n_neighbors=100) 
    umap_embeds = reducer.fit_transform(embeds)
    return (embeds, umap_embeds)

@st.cache(allow_output_mutation=True)
def get_keywords(df, n_clusters=8, chart_title='This is the title'):
  embeds, umap_embeds = get_embeddings(df)
  df['x'] = umap_embeds[:,0]
  df['y'] = umap_embeds[:,1]

  kmeans_model = KMeans(n_clusters=n_clusters, random_state=0)
  classes = kmeans_model.fit_predict(embeds)
  documents =  df['title']
  documents = pd.DataFrame({"Document": documents,
                            "ID": range(len(documents)),
                            "Topic": None})
  documents['Topic'] = classes
  documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
  count_vectorizer = CountVectorizer(stop_words="english").fit(documents_per_topic.Document)
  count = count_vectorizer.transform(documents_per_topic.Document)
  words = count_vectorizer.get_feature_names()
  ctfidf = ClassTFIDF().fit_transform(count).toarray()
  words_per_class = {label: [words[index] for index in ctfidf[label].argsort()[-10:]] for label in documents_per_topic.Topic}
  df['cluster'] = classes
  df['keywords'] = df['cluster'].map(lambda topic_num: ", ".join(np.array(words_per_class[topic_num])[:]))
  return df, chart_title

Overwriting helper.py


In [75]:
%%writefile semantic_search.py
import streamlit as st
import cohere
from datasets import load_dataset
import pandas as pd
import numpy as np
from annoy import AnnoyIndex
from helper import get_dataset, get_embeddings, get_keywords

@st.cache(allow_output_mutation=True)
def search(df, query):
  api_key = 'dwhPny8kTpkhDNpu05484MtqjFU2QKXeYx9kH6DA'
  co = cohere.Client(api_key)
  title = 'Some title needs to be inputted here'

  # Get dataset and put into dataframe
  s = load_dataset('snli')
  s.set_format('pandas')
  df = s['train'][:]

  df = get_dataset(df, text='premise', title='hypothesis')

  # Get embeddings
  embeds, umap_embeds = get_embeddings(df)
  df['x'] = umap_embeds[:,0]
  df['y'] = umap_embeds[:,1]

  # query and embedding
  temp_dict = {'text': query}
  df_query = pd.DataFrame(temp_dict, index=[0])

  # embed query
  query_embed, query_umap_embed = get_embeddings(df_query)

  # create search index
  embeds = np.array(embeds)

  search_index = AnnoyIndex(embeds.shape[1], 'angular')
  # Add all the vectors to the search index
  for i in range(len(embeds)):
      search_index.add_item(i, embeds[i])

  search_index.build(10) # 10 trees
  search_index.save('test.ann')

  # Retrieve the nearest neighbors
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                  include_distances=True)
  # Format the results
  results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                              'distance': similar_item_ids[1]})

  # find neighbours
  neighbour = []

  for k in range(len(df)):
    if k in similar_item_ids[0]:
      neighbour.append(1)
    else:
      neighbour.append(0)

  df_neighbour = pd.DataFrame(neighbour, columns=['neighbour'])

  df = df.join(df_neighbour);

  return df

Overwriting semantic_search.py


In [186]:
%%writefile app.py
# imports
import streamlit as st
from datasets import load_dataset
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
from sklearn.cluster import KMeans
from bertopic._ctfidf import ClassTFIDF
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image
import requests
from helper import get_dataset, get_embeddings, get_keywords
from semantic_search import search
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

api_key = 'WKsYz7mFBHEDsI7xsZ9KJP3WPMm5txIjKmU0eBTK'
co = cohere.Client(api_key)
title = 'Some title needs to be inputted here'

#
def main():
  s = load_dataset('snli')
  s.set_format('pandas')
  df = s['train'][:]
  df = get_dataset(df, text='premise', title='hypothesis')

  col1, mid, col2 = st.columns([1,1,10])

  col2.title('Analyze')

  image = Image.open(requests.get('https://avatars.githubusercontent.com/u/54850923?s=280&v=4', stream=True).raw)
  col1.image(image, width=80)

  app_mode = st.sidebar.selectbox('Mode', ['About', 'EDA', 'Cluster Analysis', 'Search'])

  if app_mode == "About":
      st.markdown('')

      st.markdown(
          """
          <style>
          [data-testid="stSidebar"][aria-expanded="true"] > div:first-child{
              width: 350px
          }
          [data-testid="stSidebar"][aria-expanded="false"] > div:first-child{
              width: 350px
              margin-left: -350px
          }
          </style>
          """,

          unsafe_allow_html=True,
      )
      st.markdown('')
      st.markdown("Visualize your data using Cohere's embed")
      uploaded_file = st.file_uploader("Choose a file")
      if uploaded_file is not None:
        dataframe = pd.read_csv(uploaded_file)
        st.write(dataframe)

  elif app_mode == 'EDA':
    st.sidebar.subheader(' Quick  Explore')
    st.markdown("Tick the box on the side panel to explore the dataset.")
    if st.sidebar.checkbox('Basic Info'):
        if st.sidebar.checkbox("Show Columns"):
            st.subheader('Show Columns List')
            all_columns = df.columns.to_list()
            st.write(all_columns)

        if st.sidebar.checkbox('Overview'):
            st.subheader('File contents')
            st.write(df)
        if st.sidebar.checkbox('Missing Values?'):
            st.subheader('Missing values')
            st.write(df.isnull().sum())

  elif app_mode == 'Cluster Analysis':
    df = get_dataset(df, text='review_body', title='review_title')
    #st.write(df.head())
    df, chart_title = get_keywords(df, n_clusters=8, chart_title='This is the Amazon dataset')
    selection = alt.selection_multi(fields=['keywords'], bind='legend')
    chart = alt.Chart(df).transform_calculate(
        url='https://news.ycombinator.com/item?id=' + alt.datum.id
    ).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
        x=#'x',
        alt.X('x',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labels=False, ticks=False, domain=False)
        ),
        y=
        alt.Y('y',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labels=False, ticks=False, domain=False)
        ),
        href='url:N',
        color=alt.Color('keywords:N', 
                        legend=alt.Legend(columns=1, symbolLimit=0, labelFontSize=14)
                      ),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
        tooltip=['title', 'keywords', 'cluster']
    ).properties(
        width=800,
        height=500
    ).add_selection(
        selection
    ).configure_legend(labelLimit= 0).configure_view(
        strokeWidth=0
    ).configure(background="#FAFAFA").properties(
        title=chart_title
    ).interactive()
    st.altair_chart(chart, use_container_width=True)

  elif app_mode == 'Search':
    # call the search Function
    # TODO: pass query from here
    query = st.text_input(label='Search query', value='Show me something important')

    df = search(df, query)

    # Plot
    chart = alt.Chart(df).transform_calculate(
        url='https://news.ycombinator.com/item?id=' + alt.datum.id
    ).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
        x=#'x',
        alt.X('x',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labels=False, ticks=False, domain=False)
        ),
        y=
        alt.Y('y',
            scale=alt.Scale(zero=False),
            axis=alt.Axis(labels=False, ticks=False, domain=False)
        ),
        color=alt.Color('neighbour', scale=alt.Scale(domain=[0, 1], range=['blue', 'red'])),
        tooltip=['title']
    ).properties(
        width=800,
        height=500
    ).configure_legend(labelLimit= 0).configure_view(
        strokeWidth=0
    ).configure(background="#FDF7F0")
    #chart.interactive()
    st.altair_chart(chart.interactive(), use_container_width=True)

if __name__ == '__main__':
  main()

Overwriting app.py


In [187]:
# get auth token from https://dashboard.ngrok.com/get-started/setup 

In [188]:
!streamlit run --server.port 80 app.py &>/dev/null&

In [63]:
# get auth token from https://dashboard.ngrok.com/get-started/setup 
#!ngrok config add-authtoken 2EGJ6BBj3YLPwKOmEih64tTW8Ix_2bY2s98krjdTQmjLgNYFf
!ngrok authtoken 2EGJ6BBj3YLPwKOmEih64tTW8Ix_2bY2s98krjdTQmjLgNYFf

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [66]:
from pyngrok import ngrok
public_url = ngrok.connect(port=80)
public_url

<NgrokTunnel: "http://e5f3-35-229-202-38.ngrok.io" -> "http://localhost:80">

In [65]:
# shut everything down
ngrok.kill()
ngrok.disconnect(public_url)

In [60]:
!pgrep streamlit

770


In [61]:
# insert the process number outputed by above cell
!kill 770